Link to Data: https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29

## Heart Rate
This program is how I aquired the heart rate data and windows to be used in machine learning analysis. 

In [ ]:
import scipy.signal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hrv.rri import RRi
from hrv.io import read_from_csv
import neurokit as nk
from hrv.classical import time_domain

### Getting Heart Rate Variance / Heart Rate
This function is used in the main findrrv function where it recieved a dictionary of a describe method (the method that is used to describe a dataframe called describe()) and returns a dictionary with just the important stuff (mean, median, standard deviation, and variance) for both HRV and HR

In [ ]:
def get_hrv(hrv):
    returnhrDic = {}
    returnhrvDic = {}
    
    returnhrDic['meanHr'] = hrv['mean']['hr']
    returnhrDic['medianHr'] = hrv['median']['hr']
    returnhrDic['stdHr'] = hrv['std']['hr']
    returnhrDic['varHr'] = hrv['var']['hr']
    
    
    returnhrvDic['meanHrv'] = hrv['mean']['rri']
    returnhrvDic['medianHrv'] = hrv['median']['rri']
    returnhrvDic['stdHrv'] = hrv['std']['rri']
    returnhrvDic['varHrv'] = hrv['var']['rri']
    
    return returnhrDic,returnhrvDic

### Finding Respiratory Rate Variability (heart rate)
this is the main function in this program, where I use a module called nk to retieve where the peaks of the heart rate are. Then, I use this and some math to find the intervals of these peaks and call ir hrv (heart rate variability or respiratory rate variability) I use this to find heart rate, and to gather some other useful attubitues from the time_domain function which is from the nk module. 

In [1]:
def findrrv(p):
    samplingRate = 700
    shiftStep = 175
    returnDf = pd.DataFrame()
    count = 0
    for i in range(0,len(p),shiftStep):
        try:
            bio = nk.ecg_preprocess(ecg=p[i:i+(samplingRate*60)][0], sampling_rate=samplingRate)
            #hrv = nk.bio_ecg.ecg_hrv(rpeaks=bio['ECG']['R_Peaks'], sampling_rate=sampling_rate)
            #print(bio['ECG']['R_Peaks'])
            #peakTimes = list(scipy.signal.find_peaks(p[i:i+(samplingRate*60)]['y'],100,distance = 25))[0]
            #print(peakTimes)
            peakTimes = bio['ECG']['R_Peaks']
            peakTimes = np.diff(peakTimes)
            peakTimes = peakTimes/samplingRate
            peakTimes = peakTimes.astype(float)
            hrv = RRi(peakTimes)
            hrdic,hrvdic = get_hrv(hrv.describe())
            time = time_domain(hrv)
            returnDic = dict(hrdic,**hrvdic)
            returnDic.update(time)
            temp = pd.DataFrame(returnDic,index=[i])
            returnDf = returnDf.append(temp,ignore_index=True)
            #hrv = nk.bio_ecg.ecg_hrv(rri=peakTimes, sampling_rate=sampling_rate,hrv_features=['time'])   
            count += 1
        except Exception as inst:
            print(p[i:i+(samplingRate*60)][0])
            print(inst)
    #print(count)
    return returnDf

## Respirtory Data
Here is the code that we used to gather the respirtory data as well as the temp and EMG data. It's very similar to the Heart rate gathering technic but doesn't have a fancy module do find the right peaks. For Respirtory data we just used scipy.signal's find_peaks to find the peaks of the data and say that at each peak is one breath. otherwise, it's basically the same as get data and heart rate. 

In [ ]:
import pickle
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, freqz, find_peaks
import matplotlib.pyplot as plt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data, axis=0)
    return y

def getData(sID):
    fileStr = "WESAD/WESAD/S{}/S{}.pkl".format(sID,sID)
    file = open(fileStr,'rb')
    p = pickle._Unpickler(file)
    p.encoding = 'latin1'
    u = p.load()
    data = u['signal']['chest']['Resp']
    labels = u['label']

    df = pd.DataFrame(data=data)
    df['label'] = pd.Series(labels, index=df.index)
    y = butter_lowpass_filter(data, cutoff, fs, order) # Filter data
    #y = data                                            # Don't filter data
    df['y'] = pd.DataFrame(y)
    stressDF = df[(df['label'] == 2)]

    return stressDF["y"].values

def plotData(y):
    T = 6079.0         # seconds
    n = int(T * fs) # total number of samples
    t = np.linspace(0, T, n, endpoint=False)

    plt.figure(figsize=(10,10))
    plt.subplot(2, 1, 2)
    #plt.plot(t, data, 'b-', label='data')
    plt.plot(range(len(y)), y, 'g-', linewidth=2, label='filtered data')
    plt.xlabel('Time [sec]')
    plt.ylim()
    plt.grid()
    plt.legend()

    plt.subplots_adjust(hspace=0.35)
    plt.show()

# Filter requirements.
order = 6
fs = 700.0      # sample rate, Hz
cutoff = 5  # desired cutoff frequency of the filter, Hz

# Create new data columns from the previous data
# Mean, Standard Deviation, Max, Min, Range, Slope
# id, subjID, time, Mean, Standard Deviation, Max, Min, range, slope
# 1, 2, 60, ...
# 2, 2, 120, ...
# 3, 2, 180, ...

timeFrame = 60 # 60 Seconds
timeShift = 0.25 # Shift the timeFrame up by 0.25 seconds

subjectIDs = [2]#,3,4,5,6,7,8,9,10,11,13,14,15,16,17]


# Iterate through all the subject data
for sID in subjectIDs:
    indID = 1
    outFile = open("WESAD/WESAD/S{}/S{}Respstress.csv".format(sID,sID), "w")
    outFile.write(",mean,standard_deviation,max,min,range,slope,breath_rate,in_ex_ratio,in_mean,in_std,ex_mean,ex_std,\n")

    data = getData(sID)
    #plotData(data)
    print(sID, data)

    start_t = 0
    end_t = int(fs*timeFrame)
    step = int(fs*timeShift)
    while end_t <= data.size:
        time_seg = data[start_t:end_t]

        sMean = time_seg.mean()
        sStd = np.std(time_seg)
        sMin = time_seg.min()
        sMax = time_seg.max()
        sRange = sMax - sMin
        sSlope = (time_seg[-1] - time_seg[0]) / (end_t - start_t)
        # HERE IS THE CHANGE
        iPeaks = find_peaks(time_seg,distance=1000,height=1)
        iPeakHeights = iPeaks[-1]["peak_heights"]
        #print(len(time_seg), time_seg)
        print(len(iPeaks[0]),iPeaks)

        pRate = len(iPeaks[0]) / timeFrame
        pMean_In = iPeakHeights.mean()
        pStd_In = np.std(iPeakHeights)
        num_In = len(iPeakHeights)
        # Invert the data (look at exhales)
        
        for pos in range(len(time_seg)):
            time_seg[pos] = time_seg[pos] * -1
        ePeaks = find_peaks(time_seg, distance=1000, height=1)
        ePeakHeights = ePeaks[-1]["peak_heights"]
        pStd_Ex = np.std(ePeakHeights)
        pMean_Ex = ePeakHeights.mean()
        num_Ex = len(ePeakHeights)
        # TO ABOUT HERE IS WHERE THINGS ARE DIFFERENT
        pRatio = num_In / num_Ex

        outFile.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(indID,sMean,sStd,sMax,sMin,sRange,sSlope,pRate,pRatio,pMean_In,pStd_In,pMean_Ex,pStd_Ex))

        start_t += step
        end_t += step
        indID += 1

    #plotData(data)
    outFile.close()
